In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import json
import pandas as pd
from pandas.io.json import json_normalize
MAINDIR = "./gdrive/My Drive/EA-test/"

In [0]:
#function to load json files
def openjson(file):
  with open(MAINDIR + file) as json_file:
    return json.load(json_file)

In [0]:
#Loading json files
continents = openjson('continents.json')
countries = openjson('countries.json')
languages = openjson('languages.json')

In [0]:
#Modify json file 'countries' to convert in a list of json documents that the function json_normalize will be able transform in a dataframe
countries2 = []

for key in countries:
  d = countries[key].copy()
  lgg = d['languages']
  d.update({'ID_country':key})
  d.update({'Total_languages':len(lgg)}) # To create a column to handle countries with 2 or more languages
  d.pop('languages')
  for i in lgg:
    countries2.append({**d,'ID_language':i})

In [0]:
#Convert countries2 to Dataframe
df_countries = json_normalize(countries2)
df_countries.drop_duplicates(inplace=True) #Drop duplicates if exists
df_countries.shape #Get data frame shape

(368, 9)

In [0]:
df_countries.insert(loc=0, column='ID_ctlg', value=df_countries.ID_country +df_countries.ID_language,) #Create a primary key 
df_countries.rename(columns={'name':'country_name','continent':'ID_continent'},inplace=True) # rename column 'name'
df_countries.head()

,ID_ctlg,ID_country,ID_language,Total_languages,capital,ID_continent,currency,country_name,native,phone
0,ADca,AD,ca,1,Andorra la Vella,EU,EUR,Andorra,Andorra,376
1,AEar,AE,ar,1,Abu Dhabi,AS,AED,United Arab Emirates,دولة الإمارات العربية المتحدة,971
2,AFps,AF,ps,3,Kabul,AS,AFN,Afghanistan,افغانستان,93
3,AFuz,AF,uz,3,Kabul,AS,AFN,Afghanistan,افغانستان,93
4,AFtk,AF,tk,3,Kabul,AS,AFN,Afghanistan,افغانستان,93


In [0]:
#Modify json file 'languages' to convert in a list of json documents that the function json_normalize will be able transform in a dataframe
languages2 = []

for key in languages:
  d = languages[key].copy()
  d.update({'ID_language':key})
  languages2.append(d)

In [0]:
#Convert languages2 to Dataframe
df_languages = json_normalize(languages2)
df_languages.drop_duplicates(inplace=True) #Drop duplicates if exists
df_languages.rename(columns={'name':'language_name'},inplace=True) # rename column 'name'
df_languages.head()

,ID_language,language_name,native,rtl
0,aa,Afar,Afar,NaN
1,ab,Abkhazian,Аҧсуа,NaN
2,af,Afrikaans,Afrikaans,NaN
3,ak,Akan,Akana,NaN
4,am,Amharic,አማርኛ,NaN


In [0]:
#Modify json file 'continents' to convert in a list of json documents that the function json_normalize will be able transform in a dataframe
continents2 = []
for key in continents:
  continents2.append({'ID_continent':key,'name':continents[key]})

In [0]:
#Convert continents2 to Dataframe
df_continents = json_normalize(continents2)
df_continents.drop_duplicates(inplace=True) #Drop duplicates if exists
df_continents.rename(columns={'name':'continent_name'},inplace=True) # rename column 'name'
df_continents.head()

,ID_continent,continent_name
0,AF,Africa
1,AN,Antarctica
2,AS,Asia
3,EU,Europe
4,NA,North America


In [0]:
#Join (inner) data frames, we will only use the columns related to ID and standard name of country, language and continent
#The 'ID_language' 'nb' and 'lu' are not in the data frame df_languages but these are not the principal languages of their countries 
#So we can accept  lose this information

df = df_countries.iloc[:,[0,1,2,3,5,-3]].merge(df_languages.iloc[:,:2])
df = df.merge(df_continents)
df.head()

,ID_ctlg,ID_country,ID_language,Total_languages,ID_continent,country_name,language_name,continent_name
0,ADca,AD,ca,1,EU,Andorra,Catalan,Europe
1,ESca,ES,ca,5,EU,Spain,Catalan,Europe
2,GBen,GB,en,1,EU,United Kingdom,English,Europe
3,GGen,GG,en,2,EU,Guernsey,English,Europe
4,GIen,GI,en,1,EU,Gibraltar,English,Europe


In [0]:
#save final dataframe
df.to_csv(MAINDIR + 'CompleteJSONDataset.csv',index=False)